## Install libs and import them

In [2]:
# A dependency of the preprocessing for BERT inputs

!pip install gensim
!pip install -q bertopic

In [3]:
import os
import shutil

import pandas as pd

import tensorflow as tf
import tensorflow_hub as hub
from google.colab import drive

from bertopic import BERTopic
import matplotlib.pyplot as plt

tf.get_logger().setLevel('ERROR')

/usr/local/lib/python3.12/dist-packages/hdbscan/plots.py:448: SyntaxWarning: invalid escape sequence '\l'
  axis.set_ylabel('$\lambda$ value')
/usr/local/lib/python3.12/dist-packages/hdbscan/robust_single_linkage_.py:175: SyntaxWarning: invalid escape sequence '\{'
  $max \{ core_k(a), core_k(b), 1/\alpha d(a,b) \}$.


## Topic Extraction and analysis

<!-- About topic modeling from blog-->

In this demo we use BerTopic to extract topics from a dataset, then we do some ts analysis. The dataset used in this notebook is open source and it was selected from github (https://github.com/linanqiu/reddit-dataset/tree/master). It's the reddit threads and comments dataset.  

### Load dataset

We first load the dataset from google drive and we execute some light EDA.


In [ ]:

drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
path = '/content/drive/MyDrive/Drive Blog NLP/reddit-dataset-master/learning_youshouldknow.csv'

df=pd.read_csv(path)

df.pop("Unnamed: 0")
df.columns=['text', 'id', 'subreddit', 'meta', 'time', 'author', 'ups', 'downs', 'authorlinkkarma', 'authorcommentkarma', 'authorisgold']
df

,text,id,subreddit,meta,time,author,ups,downs,authorlinkkarma,authorcommentkarma,authorisgold
0,informative post and truly helpful it s easy t...,czctg99,youshouldknow,learning,1.453833e+09,doverthere,2.0,0.0,705,4511,1.0
1,actually that is such a good tip it s not what...,czrpd2k,youshouldknow,learning,1.454896e+09,innercirclepost,6.0,0.0,1,8,0.0
2,do nt put it in the fridge with the peel on ...,czs3rfq,youshouldknow,learning,1.454931e+09,agent-99,1.0,0.0,3490,9171,0.0
3,this probably wo nt come up that often but i ...,44h2zv,youshouldknow,learning,1.454779e+09,traunks,33.0,0.0,341,1012,0.0
4,just in general with usbc be careful what you ...,czzqpho,youshouldknow,learning,1.455473e+09,xsvfan,33.0,0.0,383,25495,0.0
...,...,...,...,...,...,...,...,...,...,...,...
79473,also windows displays file sizes in kibimebig...,czc9rbl,youshouldknow,learning,1.453790e+09,[deleted],7.0,0.0,0,0,NaN
79474,deleted,czexg90,youshouldknow,learning,1.453969e+09,[deleted],-4.0,0.0,0,0,NaN
79475,there are people who do nt know this perhaps ...,cywryit,youshouldknow,learning,1.452704e+09,[deleted],1.0,0.0,0,0,NaN
79476,deleted,cys6qrw,youshouldknow,learning,1.452376e+09,[deleted],1.0,0.0,0,0,NaN


In [ ]:
df['time'] = pd.to_datetime(df['time'],unit='s')


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 79478 entries, 0 to 79477
Data columns (total 11 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   text                78678 non-null  object        
 1   id                  79478 non-null  object        
 2   subreddit           79478 non-null  object        
 3   meta                79478 non-null  object        
 4   time                78678 non-null  datetime64[ns]
 5   author              78678 non-null  object        
 6   ups                 78678 non-null  float64       
 7   downs               78678 non-null  float64       
 8   authorlinkkarma     79478 non-null  int64         
 9   authorcommentkarma  79478 non-null  int64         
 10  authorisgold        77444 non-null  float64       
dtypes: datetime64[ns](1), float64(3), int64(2), object(5)
memory usage: 6.7+ MB


In [ ]:
df.describe(include="all")

,text,id,subreddit,meta,time,author,ups,downs,authorlinkkarma,authorcommentkarma,authorisgold
count,78678,79478,79478,79478,78678,78678,78678.000000,78678.0,79478.000000,79478.000000,77444.000000
unique,1551,1591,1,1,NaN,1444,NaN,NaN,NaN,NaN,NaN
top,deleted,czctg99,youshouldknow,learning,NaN,[deleted],NaN,NaN,NaN,NaN,NaN
freq,1199,50,79478,79478,NaN,1748,NaN,NaN,NaN,NaN,NaN
mean,NaN,NaN,NaN,NaN,2016-01-26 03:23:44.901802240,NaN,64.591118,0.0,4511.927653,15623.533456,0.074686
min,NaN,NaN,NaN,NaN,2015-09-22 15:13:30,NaN,-64.000000,0.0,0.000000,-29.000000,0.000000
25%,NaN,NaN,NaN,NaN,2016-01-15 16:48:17,NaN,1.000000,0.0,38.000000,978.000000,0.000000
50%,NaN,NaN,NaN,NaN,2016-01-26 14:23:56,NaN,2.000000,0.0,475.000000,4669.000000,0.000000
75%,NaN,NaN,NaN,NaN,2016-02-04 22:45:59,NaN,10.000000,0.0,2684.000000,15954.000000,0.000000
max,NaN,NaN,NaN,NaN,2016-02-17 05:06:45,NaN,3957.000000,0.0,621795.000000,301427.000000,1.000000


In [ ]:
df.text.isna().sum()

800

In [ ]:
df.text=df.text.fillna("")

## Data Cleaning

In [ ]:
!pip install better_profanity

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.1/46.1 kB 1.9 MB/s eta 0:00:00


Comments in social media sources contains occasionally bad language. For that reason we remove such words for better topics quality.

In [ ]:
from better_profanity import profanity
df["text"] = df["text"].apply(lambda x: profanity.censor(x, ' '))


In [ ]:
df.to_csv('reddit_cleaned.csv',index=False)

if you have already cleaned the data (long cleaning process) you can skip the previous two lines and execute the following two ones instead.

In [ ]:
path="/content/drive/MyDrive/data/reddit_cleaned.csv"
df=pd.read_csv(path)

In [ ]:
df.text=df.text.fillna("")

In [ ]:
comments=df.text.unique().tolist()

We also execute some basic data cleaning by removing malformed text such as malformed domains and urls.

In [ ]:
import re

def clean_text(text):
    text = re.sub(r'\b(?:www)?[a-z0-9\-]{3,}\.(?:[a-z]{2,})?\b', '', text)
    text = re.sub(r'\b(?:www)?[a-z0-9\-]{5,}\b', '', text)
    text = re.sub(r'http\S+|www\S+', '', text)
    text = re.sub(r'\s+', ' ', text).strip()
    return text

In [ ]:
comments=[clean_text(c) for c in comments]

In [ ]:

topic_model = BERTopic( low_memory=True,calculate_probabilities=True,verbose=True)

In [ ]:
comments

['informative post and truly helpful it s easy to miss something so small as clearing your exhaust pipe  especially after you spent hours shoveling the snow and you just want it to be over',
 'actually that is such a good tip it s not what i would have thought of even though i may have done it before ',
 'do nt put it in the fridge with the peel on   it will turn black   http  indianapublicmediaorgamomentofsciencefiles200907banana940x633jpg  ',
 'this probably wo nt come up that often  but i do find sometimes i only need half of a banana i used to try to save the other half by kind of wrapping the peel around it but it never worked  here s the trick  cut the banana in half before you peel it that way the peel will stay sealed to the uneaten half you simply then just wrap saran wrap around the exposed end of the banana you do nt even have to keep it in the fridge  although that will allow it to last longer i ve done this a few times and it s remained perfectly preserved ',
 'just in gen

In [ ]:
topics, probs = topic_model.fit_transform(comments)

2025-07-31 23:25:22,761 - BERTopic - Embedding - Transforming documents to embeddings.


Batches:   0%|          | 0/49 [00:00<?, ?it/s]

2025-07-31 23:26:10,978 - BERTopic - Embedding - Completed ✓
2025-07-31 23:26:10,979 - BERTopic - Dimensionality - Fitting the dimensionality reduction algorithm
2025-07-31 23:26:15,497 - BERTopic - Dimensionality - Completed ✓
2025-07-31 23:26:15,498 - BERTopic - Cluster - Start clustering the reduced embeddings
2025-07-31 23:26:15,736 - BERTopic - Cluster - Completed ✓
2025-07-31 23:26:15,742 - BERTopic - Representation - Fine-tuning topics using representation models.
2025-07-31 23:26:15,862 - BERTopic - Representation - Completed ✓


## Model evaluation
Checking topics coherence from the trained bert model
- first use gensim evaluation metrics to compare , then use simplified pipeline to optimise hyperparams
- second use this lib : https://github.com/MIND-Lab/OCTIS for hyper params optimisation

In [ ]:
from gensim import corpora
from gensim.models.coherencemodel import CoherenceModel
#method found in : https://github.com/MaartenGr/BERTopic/issues/469
def calculate_coherence_metrics(topic_model,comments,topics):
  metrics={}
  documents = pd.DataFrame({"comment": comments,
                            "id": range(len(comments)),
                            "topic": topics})
  documents_per_topic = documents.groupby(['topic'], as_index=False).agg({'comment': ' '.join})
  cleaned_docs = topic_model._preprocess_text(documents_per_topic.comment.values)
  vectorizer = topic_model.vectorizer_model
  analyzer = vectorizer.build_analyzer()
  words = vectorizer.get_feature_names_out()
  tokens = [analyzer(doc) for doc in cleaned_docs]
  dictionary = corpora.Dictionary(tokens)
  corpus = [dictionary.doc2bow(token) for token in tokens]
  topic_words = [[words for words, _ in topic_model.get_topic(topic)] for topic in range(len(set(topics))-1)]


  metrics["u_mass_coherence"]=CoherenceModel(topics=topic_words,
                                 texts=tokens,  corpus=corpus,
                                 dictionary=dictionary, coherence='u_mass').get_coherence()
  metrics["c_v_coherence"]=CoherenceModel(topics=topic_words,
                                 texts=tokens,  corpus=corpus,
                                 dictionary=dictionary, coherence='c_v').get_coherence()
  metrics["c_uci_coherence"]=CoherenceModel(topics=topic_words,
                                 texts=tokens,  corpus=corpus,
                                 dictionary=dictionary, coherence='c_uci').get_coherence()
  metrics["c_npmi_coherence"]=CoherenceModel(topics=topic_words,
                                 texts=tokens,  corpus=corpus,
                                 dictionary=dictionary, coherence='c_npmi').get_coherence()
  return metrics

In [ ]:
coherence_metrics=calculate_coherence_metrics(topic_model,comments,topics)

In [ ]:
coherence_metrics

{'u_mass_coherence': -0.5423984897540505,
 'c_v_coherence': 0.6382336416341199,
 'c_uci_coherence': -2.6242854186847,
 'c_npmi_coherence': 0.007330986080360313}

In [ ]:
topic_model.get_topic_info()

,Topic,Count,Name,Representation,Representative_Docs
0,-1,297,-1_you_to_it_this,"[you, to, it, this, on, the, in, and, that, for]",[just to be clear i m not a windows 10 hater ...
1,0,133,0_free_tax_taxes_you,"[free, tax, taxes, you, for, file, and, pay, t...",[you can file your taxes for free anyway but ...
2,1,94,1_this_thanks_thank_know,"[this, thanks, thank, know, great, why, op, no...","[this is , thanks so much for this , than..."
3,2,85,2_she_her_my_spot,"[she, her, my, spot, disabled, he, and, parkin...",[i also get looks when i take my disabled moth...
4,3,74,3_doctor_and_to_doctors,"[doctor, and, to, doctors, nt, they, had, my, ...",[a 2nd opinion has saved or prolonged a life t...
5,4,65,4_library_text_card_it,"[library, text, card, it, wikipedia, phone, th...",[ kiwix http wwwkiwixorgwikimain_page is a...
6,5,46,5_antivirus_bitdefender_avast_the,"[antivirus, bitdefender, avast, the, free, of,...",[ tl dr if you are running windows and need ...
7,6,43,6_ethanol_gas_fuel_water,"[ethanol, gas, fuel, water, gasoline, engine, ...",[the problem with gas is the short shelf life ...
8,7,40,7_drive_backup_data_up,"[drive, backup, data, up, back, files, my, to,...",[i make sure i have at least two copies of eve...
9,8,38,8_cold_sore_abreva_sores,"[cold, sore, abreva, sores, mouth, the, lip, i...",[i have a sure remedy and guy s and s it ...


As we can see, the model's default representation output might not be easy to interpret: many articles and non-representative keywords appear in the topics.  Fine-tuning a custom representation model is necessary for our use case to generate more meaningful topics. For this purpose, we use KeyBERT to diversify the topic keywords and enhance their interpretability.

In [ ]:
from bertopic.representation import KeyBERTInspired
# Create your representation model
representation_model = KeyBERTInspired()

topic_model_improved = BERTopic( low_memory=True,calculate_probabilities=True,verbose=True,representation_model=representation_model)

In [ ]:
topics_n, probs_n=topic_model_improved.fit_transform(comments)

2025-07-31 23:28:31,874 - BERTopic - Embedding - Transforming documents to embeddings.


Batches:   0%|          | 0/49 [00:00<?, ?it/s]

2025-07-31 23:29:17,226 - BERTopic - Embedding - Completed ✓
2025-07-31 23:29:17,228 - BERTopic - Dimensionality - Fitting the dimensionality reduction algorithm
2025-07-31 23:29:23,656 - BERTopic - Dimensionality - Completed ✓
2025-07-31 23:29:23,657 - BERTopic - Cluster - Start clustering the reduced embeddings
2025-07-31 23:29:23,876 - BERTopic - Cluster - Completed ✓
2025-07-31 23:29:23,882 - BERTopic - Representation - Fine-tuning topics using representation models.
2025-07-31 23:29:29,959 - BERTopic - Representation - Completed ✓


In [ ]:
topic_model_improved.get_topics()

{-1: [('how', 0.26784837),
  ('using', 0.19766784),
  ('use', 0.1852465),
  ('have', 0.16990498),
  ('know', 0.16589744),
  ('that', 0.1637072),
  ('do', 0.16245279),
  ('just', 0.16064931),
  ('never', 0.15858206),
  ('by', 0.15425947)],
 0: [('https', 0.23767155),
  ('that', 0.22330272),
  ('this', 0.21886063),
  ('http', 0.21800631),
  ('simple', 0.19971004),
  ('easy', 0.19599748),
  ('can', 0.1937404),
  ('more', 0.19112921),
  ('of', 0.18373194),
  ('like', 0.18029198)],
 1: [('have', 0.26218718),
  ('isnt', 0.25237232),
  ('found', 0.24102579),
  ('for', 0.23222923),
  ('tip', 0.22631001),
  ('do', 0.22384085),
  ('post', 0.22274187),
  ('then', 0.22171307),
  ('if', 0.21932694),
  ('again', 0.21802372)],
 2: [('not', 0.4960704),
  ('for', 0.4922192),
  ('nt', 0.49026814),
  ('can', 0.4780009),
  ('this', 0.46299338),
  ('that', 0.46245056),
  ('so', 0.4600702),
  ('is', 0.45229894),
  ('to', 0.45190647),
  ('but', 0.45001042)],
 3: [('as', 0.19155703),
  ('if', 0.18652786),
  (

In [ ]:
coherence_metrics=calculate_coherence_metrics(topic_model_improved,comments,topics_n)
coherence_metrics

{'u_mass_coherence': -0.7269715747364682,
 'c_v_coherence': 0.3551674363004895,
 'c_uci_coherence': -6.294601969755953,
 'c_npmi_coherence': -0.21081947605182805}

The lower coherence observed with the centroid-based representation is expected, as KeyBERT is designed to enhance topic quality in BERTopic by providing more meaningful and diverse keywords.

In [ ]:
topic_model_improved.get_topic_info()

,Topic,Count,Name,Representation,Representative_Docs
0,-1,337,-1_how_using_use_have,"[how, using, use, have, know, that, do, just, ...",[why should i know this i believe it s better...
1,0,127,0_https_that_this_http,"[https, that, this, http, simple, easy, can, m...",[you can file your taxes for free anyway but ...
2,1,99,1_have_isnt_found_for,"[have, isnt, found, for, tip, do, post, then, ...","[this is , thanks so much for this , than..."
3,2,84,2_not_for_nt_can,"[not, for, nt, can, this, that, so, is, to, but]",[i also get looks when i take my disabled moth...
4,3,74,3_as_if_take_for,"[as, if, take, for, did, with, do, nt, just, w...",[a 2nd opinion has saved or prolonged a life t...
5,4,40,4_not_up_now_fine,"[not, up, now, fine, you, should, will, just, ...",[the problem with gas is the short shelf life ...
6,5,40,5_that_about_and_re,"[that, about, and, re, what, good, is, you, if...",[ tl dr if you are running windows and need ...
7,6,38,6_that_works_on_you,"[that, works, on, you, into, this, to, out, us...",[i have a sure remedy and guy s and s it ...
8,7,38,7_restore_nt_321_version,"[restore, nt, 321, version, got, not, use, fol...",[i make sure i have at least two copies of eve...
9,8,37,8_and_it_add_10x,"[and, it, add, 10x, 10, that, sum, same, then,...",[explanation of the 7 rule since it took ...


In [ ]:
topic_model_improved.get_document_info(comments)

,Document,Topic,Name,Representation,Representative_Docs,Top_n_words,Probability,Representative_document
0,informative post and truly helpful it s easy t...,32,32_ticketing_wwwnydailynewscomnewyorkbrooklynb...,"[ticketing, wwwnydailynewscomnewyorkbrooklynbr...","[i m from canada what s a snow emergency , wit...",ticketing - wwwnydailynewscomnewyorkbrooklynbr...,0.344923,False
1,actually that is such a good tip it s not what...,1,1_have_isnt_found_for,"[have, isnt, found, for, tip, do, post, then, ...","[this is , thanks so much for this , than...",have - isnt - found - for - tip - do - post - ...,0.597480,False
2,do nt put it in the fridge with the peel on ...,27,27_http_need_titletext_mentalflosscomarticle31...,"[http, need, titletext, mentalflosscomarticle3...",[ysk ozone destroy ethylene gas helping keep f...,http - need - titletext - mentalflosscomarticl...,0.137166,False
3,this probably wo nt come up that often but i ...,27,27_http_need_titletext_mentalflosscomarticle31...,"[http, need, titletext, mentalflosscomarticle3...",[ysk ozone destroy ethylene gas helping keep f...,http - need - titletext - mentalflosscomarticl...,1.000000,True
4,just in general with usbc be careful what you ...,-1,-1_how_using_use_have,"[how, using, use, have, know, that, do, just, ...",[why should i know this i believe it s better...,how - using - use - have - know - that - do - ...,0.701761,False
...,...,...,...,...,...,...,...,...
1547,so if 73 are nt installed properly is there ...,22,22_nt_ie_manual_safe,"[nt, ie, manual, safe, local, mind, any, them,...",[also for children around two and older a p...,nt - ie - manual - safe - local - mind - any -...,1.000000,False
1548,certified child passenger safety technician he...,22,22_nt_ie_manual_safe,"[nt, ie, manual, safe, local, mind, any, them,...",[also for children around two and older a p...,nt - ie - manual - safe - local - mind - any -...,1.000000,False
1549,or you can vent all your problems right here o...,-1,-1_how_using_use_have,"[how, using, use, have, know, that, do, just, ...",[why should i know this i believe it s better...,how - using - use - have - know - that - do - ...,0.757225,False
1550,i think this is usa only yes it s similar to ...,-1,-1_how_using_use_have,"[how, using, use, have, know, that, do, just, ...",[why should i know this i believe it s better...,how - using - use - have - know - that - do - ...,0.381640,False


We now use lemmatisation to avoid repeating keywords in the topics having the same root.


In [ ]:
from nltk import word_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import CountVectorizer
import nltk
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('punkt_tab')

class LemmaTokenizer:
    def __init__(self):
        self.wnl = WordNetLemmatizer()
    def __call__(self, doc):
        return [self.wnl.lemmatize(t) for t in word_tokenize(doc)]
vectorizer_model= CountVectorizer(tokenizer=LemmaTokenizer())

lemma_topic_model = BERTopic(vectorizer_model=vectorizer_model)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


In [ ]:
topics_nl, probs_nl=lemma_topic_model.fit_transform(comments)

In [ ]:
coherence_metrics=calculate_coherence_metrics(lemma_topic_model,comments,topics_nl)
print(coherence_metrics)
lemma_topic_model.get_topic_info()

{'u_mass_coherence': -0.5560704088811599, 'c_v_coherence': 0.6176426199262081, 'c_uci_coherence': -2.2208819351595697, 'c_npmi_coherence': 0.013289891356831302}


,Topic,Count,Name,Representation,Representative_Docs
0,-1,276,-1_you_to_it_a,"[you, to, it, a, i, the, this, on, and, that]",[in any new relationship people are infatuate...
1,0,130,0_tax_free_you_for,"[tax, free, you, for, and, year, pay, turbotax...",[many of us have been paying turbo tax or hamp...
2,1,115,1_this_thanks_thank_know,"[this, thanks, thank, know, great, why, someth...","[this is , thanks so much for this , than..."
3,2,82,2_she_her_i_a,"[she, her, i, a, my, spot, disabled, and, he, to]",[when i broke my ankle i got a temporary plac...
4,3,75,3_doctor_and_to_a,"[doctor, and, to, a, i, nt, they, that, my, had]",[this reminds me of a story i read by richard ...
5,4,72,4_text_library_book_it,"[text, library, book, it, i, card, this, phone...",[ kiwix http wwwkiwixorgwikimain_page is a...
6,5,42,5_gas_ethanol_engine_fuel,"[gas, ethanol, engine, fuel, water, gasoline, ...",[i work at a lawn equipment shop and around 80...
7,6,42,6_antivirus_bitdefender_avast_the,"[antivirus, bitdefender, avast, the, of, is, a...",[ tl dr if you are running windows and need ...
8,7,38,7_number_by_divisible_7,"[number, by, divisible, 7, digit, 2, rule, 11,...",[gt if you do nt know then halve the last 2 d...
9,8,36,8_sore_cold_abreva_mouth,"[sore, cold, abreva, mouth, the, lip, it, i, a...",[i have a sure remedy and guy s and s it ...


The final step of this model training process is to select the best parameters for the different models. To do so, we need to compare the topic-building pipelines and identify the one that represents the optimal set of hyperparameters.

In practice, for unsupervised models in general ,and topic inference in particular, the metrics are not 100% reliable, and human judgment is required. For that reason, we select the top 5 models and make a final decision after visualizing the results.

In [ ]:
from sentence_transformers import SentenceTransformer
from umap import UMAP
from sklearn.decomposition import PCA
from hdbscan import HDBSCAN
from sklearn.cluster import KMeans
from bertopic.vectorizers import ClassTfidfTransformer
from transformers.pipelines import pipeline
import tensorflow_hub
##hyperparameters tuning

#pipeline 1
representation_model = KeyBERTInspired()

p1 = BERTopic( representation_model=representation_model
              ,low_memory=True,calculate_probabilities=True,verbose=True)

#pipeline 2
vectorizer_model= CountVectorizer(tokenizer=LemmaTokenizer())

p2 = BERTopic(vectorizer_model=vectorizer_model
              ,low_memory=True,calculate_probabilities=True,verbose=True)

#pipelines 3
#---embedding extractors
extractors=[ SentenceTransformer("all-MiniLM-L6-v2"),
            #pipeline("feature-extraction", model="distilbert-base-cased"),
            tensorflow_hub.load("https://tfhub.dev/google/universal-sentence-encoder/4")]
#---dimentiality reduction
dim_red=[ UMAP(n_neighbors=15, n_components=10, min_dist=0.0, metric='cosine'),
         UMAP(n_neighbors=20, n_components=15, min_dist=0.0, metric='euclidean'),
         PCA(n_components=5),
         PCA(n_components=10)]
#---clustering
clus=[HDBSCAN(min_cluster_size=10, metric='euclidean', cluster_selection_method='eom', prediction_data=True),
HDBSCAN(min_cluster_size=10, metric='manhattan', cluster_selection_method='eom', prediction_data=True),
HDBSCAN(min_cluster_size=5, metric='euclidean', cluster_selection_method='eom', prediction_data=True),
KMeans(n_clusters=20)
]
#---bag-of-words extraction
vectorisers=[CountVectorizer(stop_words="english"),
             CountVectorizer(tokenizer=LemmaTokenizer())
             ,CountVectorizer(tokenizer=LemmaTokenizer(),stop_words="english")
             ,CountVectorizer(tokenizer=LemmaTokenizer(),
              ngram_range=(1, 2),stop_words="english")
             ,CountVectorizer(tokenizer=LemmaTokenizer(),
              ngram_range=(2, 2),stop_words="english")]
#---topic representation
reps=[
    #ClassTfidfTransformer(),
    KeyBERTInspired()]
general_params =[]

In [ ]:
meta=[]
pipelines=[]
for e in extractors:
  for d in dim_red:
    for c in clus:
      for v in vectorisers:
        for r in reps:
          meta.append({"Topic extraction": str(e),
                       "Dimensionality Reduction":str(d),
                       "Clustering":str(c),
                       "Vectorization":str(v),
                       "Representation":str(r)})
          pipelines.append(BERTopic(  embedding_model=e,
                                      umap_model=d,
                                      hdbscan_model=c,
                                      vectorizer_model=v,
                                      representation_model=r,
                                      low_memory=True,
                                      calculate_probabilities=True,
                                      verbose=False))


In [ ]:
i=0
metrics = {
    'Run': [],
    'Model': [],
    'u_mass_coherence': [],
    'c_v_coherence': []
}

df_metrics1 = pd.DataFrame(metrics)
for p in pipelines:

  print(f'-------RUN{i}-------')
  print(meta[i])
  i=i+1
  print(p)
  topics, _ = p.fit_transform(comments)

  coherence_metrics=calculate_coherence_metrics(p,comments,topics)

  df_metrics1 = pd.concat([df_metrics1, pd.DataFrame([{'Run':i,'Model':p,'u_mass_coherence': coherence_metrics['u_mass_coherence'],'c_v_coherence': coherence_metrics['c_v_coherence']}])], ignore_index=True)
df_metrics1

-------RUN0-------
{'Topic extraction': "SentenceTransformer(\n  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: BertModel \n  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})\n  (2): Normalize()\n)", 'Dimensionality Reduction': "UMAP(metric='cosine', min_dist=0.0, n_components=10)", 'Clustering': 'HDBSCAN(min_cluster_size=10, prediction_data=True)', 'Vectorization': "CountVectorizer(stop_words='english')", 'Representation': 'KeyBERTInspired()'}
BERTopic(calculate_probabilities=True, ctfidf_model=ClassTfidfTransformer(...), embedding_model=SentenceTransformer(...), hdbscan_model=HDBSCAN(...), language=None, low_memory=True, min_topic_size=10, n_gram_range=(1, 1), nr_topics=None, representation_model=KeyBE

/usr/local/lib/python3.11/dist-packages/numpy/core/fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/usr/local/lib/python3.11/dist-packages/numpy/core/_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


-------RUN13-------
{'Topic extraction': "SentenceTransformer(\n  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: BertModel \n  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})\n  (2): Normalize()\n)", 'Dimensionality Reduction': "UMAP(metric='cosine', min_dist=0.0, n_components=10)", 'Clustering': 'HDBSCAN(prediction_data=True)', 'Vectorization': "CountVectorizer(ngram_range=(1, 2), stop_words='english',\n                tokenizer=<__main__.LemmaTokenizer object at 0x7df3b891dd10>)", 'Representation': 'KeyBERTInspired()'}
BERTopic(calculate_probabilities=True, ctfidf_model=ClassTfidfTransformer(...), embedding_model=SentenceTransformer(...), hdbscan_model=HDBSCAN(...), language=None, low_memory=True, min

/usr/local/lib/python3.11/dist-packages/numpy/core/fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/usr/local/lib/python3.11/dist-packages/numpy/core/_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


-------RUN14-------
{'Topic extraction': "SentenceTransformer(\n  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: BertModel \n  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})\n  (2): Normalize()\n)", 'Dimensionality Reduction': "UMAP(metric='cosine', min_dist=0.0, n_components=10)", 'Clustering': 'HDBSCAN(prediction_data=True)', 'Vectorization': "CountVectorizer(ngram_range=(2, 2), stop_words='english',\n                tokenizer=<__main__.LemmaTokenizer object at 0x7df4a9008410>)", 'Representation': 'KeyBERTInspired()'}
BERTopic(calculate_probabilities=True, ctfidf_model=ClassTfidfTransformer(...), embedding_model=SentenceTransformer(...), hdbscan_model=HDBSCAN(...), language=None, low_memory=True, min

...

/usr/local/lib/python3.11/dist-packages/numpy/core/fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/usr/local/lib/python3.11/dist-packages/numpy/core/_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


-------RUN113-------
{'Topic extraction': '<tensorflow.python.saved_model.load.Loader._recreate_base_user_object.<locals>._UserObject object at 0x7df4a91f6e50>', 'Dimensionality Reduction': 'UMAP(min_dist=0.0, n_components=15, n_neighbors=20)', 'Clustering': 'HDBSCAN(prediction_data=True)', 'Vectorization': "CountVectorizer(ngram_range=(1, 2), stop_words='english',\n                tokenizer=<__main__.LemmaTokenizer object at 0x7df3b891dd10>)", 'Representation': 'KeyBERTInspired()'}
BERTopic(calculate_probabilities=True, ctfidf_model=ClassTfidfTransformer(...), embedding_model=<tensorflow.python.saved_model.load.Loader._recreate_base_user_object.<locals>._UserObject object at 0x7df4a91f6e50>, hdbscan_model=HDBSCAN(...), language=None, low_memory=True, min_topic_size=10, n_gram_range=(1, 1), nr_topics=None, representation_model=KeyBERTInspired(...), seed_topic_list=None, top_n_words=10, umap_model=UMAP(...), vectorizer_model=CountVectorizer(...), verbose=False, zeroshot_min_similarity=0

/usr/local/lib/python3.11/dist-packages/numpy/core/fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/usr/local/lib/python3.11/dist-packages/numpy/core/_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


-------RUN114-------
{'Topic extraction': '<tensorflow.python.saved_model.load.Loader._recreate_base_user_object.<locals>._UserObject object at 0x7df4a91f6e50>', 'Dimensionality Reduction': 'UMAP(min_dist=0.0, n_components=15, n_neighbors=20)', 'Clustering': 'HDBSCAN(prediction_data=True)', 'Vectorization': "CountVectorizer(ngram_range=(2, 2), stop_words='english',\n                tokenizer=<__main__.LemmaTokenizer object at 0x7df4a9008410>)", 'Representation': 'KeyBERTInspired()'}
BERTopic(calculate_probabilities=True, ctfidf_model=ClassTfidfTransformer(...), embedding_model=<tensorflow.python.saved_model.load.Loader._recreate_base_user_object.<locals>._UserObject object at 0x7df4a91f6e50>, hdbscan_model=HDBSCAN(...), language=None, low_memory=True, min_topic_size=10, n_gram_range=(1, 1), nr_topics=None, representation_model=KeyBERTInspired(...), seed_topic_list=None, top_n_words=10, umap_model=UMAP(...), vectorizer_model=CountVectorizer(...), verbose=False, zeroshot_min_similarity=0

,Run,Model,u_mass_coherence,c_v_coherence
0,1.0,"BERTopic(calculate_probabilities=True, ctfidf_...",-0.817523,0.464426
1,2.0,"BERTopic(calculate_probabilities=True, ctfidf_...",-0.621999,0.361650
2,3.0,"BERTopic(calculate_probabilities=True, ctfidf_...",-0.786082,0.466844
3,4.0,"BERTopic(calculate_probabilities=True, ctfidf_...",-0.709494,0.497112
4,5.0,"BERTopic(calculate_probabilities=True, ctfidf_...",-0.221516,0.478240
...,...,...,...,...
144,156.0,"BERTopic(calculate_probabilities=True, ctfidf_...",-0.672964,0.367068
145,157.0,"BERTopic(calculate_probabilities=True, ctfidf_...",-0.491039,0.400427
146,158.0,"BERTopic(calculate_probabilities=True, ctfidf_...",-0.625908,0.376987
147,159.0,"BERTopic(calculate_probabilities=True, ctfidf_...",-0.592832,0.469385


In [ ]:
df_metrics1

In [ ]:
print(meta[4])

In [ ]:
mymodel=df_metrics1.iloc[4]["Model"]

In [ ]:
df_metrics1.sort_values(by=['c_v_coherence'], ascending=False).head(20)

From the above results we select the model with the highest c_v_coherence and with a mass_coherence close to zero. choosing the best configuration is a tradeoff between those metrics but the final decision is based human judgement after data visualization.

In [ ]:
mymodel.hdbscan_model

HDBSCAN(min_cluster_size=10, prediction_data=True)

In [ ]:
mymodel.min_topic_size=15
mymodel.fit_transform(comments)

## Topics visualization

In [ ]:
mymodel.get_document_info(comments)

,Document,Topic,Name,Representation,Representative_Docs,Top_n_words,Probability,Representative_document
0,informative post and truly helpful it s easy t...,5,5_wwwsrhnoaagovffc_http wwwsrhnoaagovffc_help ...,"[wwwsrhnoaagovffc, http wwwsrhnoaagovffc, help...",[with the impending northeast storm there s a...,wwwsrhnoaagovffc - http wwwsrhnoaagovffc - hel...,0.532638,False
1,actually that is such a good tip it s not what...,1,1_rbreadit good_rbreadit_post written_thanks tip,"[rbreadit good, rbreadit, post written, thanks...","[thanks op , i totally forgot goggle my drive ...",rbreadit good - rbreadit - post written - than...,0.462085,False
2,do nt put it in the fridge with the peel on ...,28,28_peel turn_peel way_s_overripening spoiling,"[peel turn, peel way, s, overripening spoiling...",[ysk ozone destroy ethylene gas helping keep f...,peel turn - peel way - s - overripening spoili...,0.165413,False
3,this probably wo nt come up that often but i ...,28,28_peel turn_peel way_s_overripening spoiling,"[peel turn, peel way, s, overripening spoiling...",[ysk ozone destroy ethylene gas helping keep f...,peel turn - peel way - s - overripening spoili...,1.000000,True
4,just in general with usbc be careful what you ...,-1,-1_advice_try_remindme_scrap,"[advice, try, remindme, scrap, cheap, copy, us...",[just as there is a difference between the com...,advice - try - remindme - scrap - cheap - copy...,0.412432,False
...,...,...,...,...,...,...,...,...
1547,so if 73 are nt installed properly is there ...,24,24_good_check_used_nt,"[good, check, used, nt, checked, better, sure,...",[also for children around two and older a p...,good - check - used - nt - checked - better - ...,1.000000,False
1548,certified child passenger safety technician he...,24,24_good_check_used_nt,"[good, check, used, nt, checked, better, sure,...",[also for children around two and older a p...,good - check - used - nt - checked - better - ...,1.000000,False
1549,or you can vent all your problems right here o...,-1,-1_advice_try_remindme_scrap,"[advice, try, remindme, scrap, cheap, copy, us...",[just as there is a difference between the com...,advice - try - remindme - scrap - cheap - copy...,0.719516,False
1550,i think this is usa only yes it s similar to ...,4,4_inactive fix_noticed_download_question,"[inactive fix, noticed, download, question, gr...",[ kiwix http wwwkiwixorgwikimain_page is a...,inactive fix - noticed - download - question -...,0.039347,False


In [ ]:
mymodel.visualize_topics()

In [ ]:
# Visualize top topic keywords
mymodel.visualize_barchart(top_n_topics=12)

In [ ]:
mymodel.visualize_hierarchy()

In [ ]:
mymodel.visualize_heatmap()

In [ ]:
mymodel.visualize_term_rank()

In [ ]:
from sentence_transformers import SentenceTransformer
sentence_model = SentenceTransformer("all-MiniLM-L6-v2")
embeddings = sentence_model.encode(comments, show_progress_bar=True)

Batches:   0%|          | 0/49 [00:00<?, ?it/s]

In [ ]:
mymodel.save("/content/drive/MyDrive/Drive Blog NLP/best_model_reddit_bertopic_14_Juin")

In [ ]:
my_model = BERTopic.load("/content/drive/MyDrive/Drive Blog NLP/my_model_reddit_bertopic")